In [ ]:
#%pip install pytesseract

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Imports

In [2]:
import pytesseract
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pytesseract
from torchvision import transforms

### DataLoader with Optical Character Recognition (OCR): 
To load train and test data after extracting text from images using Tesseract

In [ ]:
class CustomOCRDataset (Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths

        #Transformations done post tesseract processing
        self.transform = transform
    
    def __getitem__(self, index):
        img_path = self.image_paths[index]

        #Load image as PIL and ensure RGB format
        image = Image.open(img_path).convert('RGB')

        #Add resizing and other preprocesssing as needed below
        # -
        # -
        # -

        #Extracting OCR text
        ocr_txt = pytesseract.image_to_string(image)

        return image, ocr_txt
    
    def __len__(self):
        #count of images
        return len(self.image_paths)



In [ ]:
image_path = "E:\\PROJECTS\\Personal Projects\\AI-Powered-Contract-Auditing-System\\Datasets\\SignverOD\\images"  
dataset = CustomOCRDataset(image_path, transform=None)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)